In [29]:
import pandas as pd
import plotly.express as px
import re
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

from DatabaseLocator import DatabaseLocator

In [30]:
Locator = DatabaseLocator()
netzero1 = pd.read_csv(filepath_or_buffer=Locator.get_onedrive_path("data/netzero-data/All-Subsystems-minute.csv"))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/bd/t85lw5j14_vc55x5tn1xbmr00000gn/T/ipykernel_73147/3672443998.py", line 2, in <module>
    netzero1 = pd.read_csv(filepath_or_buffer=Locator.get_onedrive_path("data/netzero-data/All-Subsystems-minute.csv"))
  File "/usr/local/lib/python3.9/site-packages/pandas/util/_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.9/site-packages/pandas/io/parsers/readers.py", line 586, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.9/site-packages/pandas/io/parsers/readers.py", line 488, in _read
    return parser.read(nrows)
  File "/usr/local/lib/python3.9/site-packages/pandas/io/parsers/readers.py", line 1047, in read
    index, columns, col_dict = self._engine.read(nrows)
  File "/usr/local/lib/p

TypeError: object of type 'NoneType' has no len()

In [ ]:
netzero2 = pd.read_csv(filepath_or_buffer=Locator.get_onedrive_path("data/netzero-data/All-Subsystems-minute (1).csv"))

In [ ]:
metadata = pd.read_csv(filepath_or_buffer=Locator.get_onedrive_path("data/netzero-data/Metadata-minute.csv"))

In [ ]:
netzero1["Timestamp"] = pd.to_datetime(netzero1["Timestamp"], format="%Y-%m-%d %H:%M:%S%z")

In [ ]:
netzero1.loc[:50]

In [ ]:
netzero2.loc[:50]

In [ ]:
netzero1.describe().T

In [ ]:
netzero2.columns.difference(netzero1.columns)

In [ ]:
netzero1.columns[3]

In [ ]:
netzero1.columns.tolist()

In [ ]:
totals = [x for x in netzero1.columns if "Total" in x]
totals

In [ ]:
fig = px.scatter(netzero1, x="Timestamp", y="Load_KPlugLoadsPowerUsage", color="Load_StatusPlugLoadToaster")
fig.show()

In [ ]:
plugs = metadata.loc[(metadata['Parameter'] == "Status_OnOff") & (metadata["Measurement_Location"] == "Kitchen") & (
    metadata["Unnamed: 0"].str.contains("Plug"))]["Unnamed: 0"].tolist()  #

In [ ]:
kitchenplugs = netzero1.loc[0:10000, ["Timestamp", "Load_KPlugLoadsPowerUsage"] + plugs].copy()
kitchenplugs["Status"] = kitchenplugs[plugs].sum(axis=1)

In [ ]:
(kitchenplugs[plugs] < 0) | (kitchenplugs[plugs] > 1)

In [ ]:
appliance_dict = {}
for appliance in plugs:
    name = " ".join(re.findall('[A-Z][^A-Z]*', appliance.split("Load")[-1]))
    appliance_switch = kitchenplugs[(kitchenplugs[appliance] != kitchenplugs[appliance].shift(1))][
                           ["Timestamp", appliance]][1:]
    appliance_dict[name] = pd.DataFrame(
        {"Timestamp": [appliance_switch.loc[index, "Timestamp"] for index in appliance_switch.index],
         "Appliance": name,
         "Color": ["green" if appliance_switch.loc[index, appliance] == 1 else "red" for index in
                   appliance_switch.index]})

appliance_dict

In [61]:
fig = px.scatter(kitchenplugs, x=[time.timestamp()*1000 for time in kitchenplugs["Timestamp"]], y="Load_KPlugLoadsPowerUsage", color="Status")
for appliance, df in appliance_dict.items():
    for index in df.index:
        fig.add_vline(x=df.loc[index, "Timestamp"].timestamp()*1000, line_width=3, line_dash="dash",
                      annotation_text=df.loc[index, "Appliance"], line_color=df.loc[index, "Color"])  #

# fig.update_layout(
#     xaxis_tickformat = "yyyy-mm-dd HH:MM:SS"
# )

fig.update_layout(
            xaxis = {'type': 'date','tickformat': '%Y-%m-%d %H:%M:%S'},
            annotations=[{**a, **{"y":.5}}  for a in fig.to_dict()["layout"]["annotations"]]
            )

fig.update_layout(
    xaxis_tickformatstops = [
        dict(dtickrange=[None, 60000], value="%H:%M:%S s"),
        dict(dtickrange=[60000, 3600000], value="%H:%M m"),
        dict(dtickrange=[3600000, 86400000], value="%H:%M h"),
        dict(dtickrange=[86400000, 604800000], value="%e. %b d"),
        dict(dtickrange=[604800000, "M1"], value="%e. %b w"),
        dict(dtickrange=["M1", "M12"], value="%b '%y M"),
        dict(dtickrange=["M12", None], value="%Y Y")
    ]
)


fig.show()

In [ ]:
kitchenplugs[kitchenplugs["Load_StatusPlugLoadToasterOven"] == 1][["Timestamp", "Load_StatusPlugLoadToasterOven"]]

In [ ]:
fig = px.scatter(kitchenplugs, x="Timestamp",
                 y=[kitchenplugs["Load_StatusPlugLoadToasterOven" == 1], "Load_StatusPlugLoadBlender"])
fig.show()